In [ ]:
## REMEMBER TO ACTIVATE `GPU_optimization (aGPUo)`

import tensorflow
import torch
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scienceplots

from matplotlib import animation
from matplotlib.animation import PillowWriter

plt.style.use(['science', 'notebook'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())